In [ ]:
#check GPU and CUDA
!nvidia-smi

In [ ]:
#get current working directory
import os
os.getcwd()

In [ ]:
#install necessary packages
%pip install scikit-optimize
%pip install opencv-python-headless
%pip install seaborn
%pip install umap-learn
%pip install openpyxl

In [ ]:
#import other libraries
# DO "pip install [package-name]" for libraries suiting your requirements
import zipfile
import os
import warnings 
warnings.filterwarnings('ignore',category=FutureWarning) 
import struct
import zlib
import time
import pickle
import itertools
from itertools import cycle
from matplotlib import pyplot
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss
from numpy import sqrt
from numpy import argmax
from numpy import genfromtxt
import pandas as pd
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import VGG16, DenseNet121, ResNet50, MobileNet, EfficientNetB0, InceptionV3
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Dropout, GlobalAveragePooling2D, Dense,Conv2D, SeparableConv2D
from sklearn.metrics import roc_curve, auc,  precision_recall_curve, average_precision_score, matthews_corrcoef
from sklearn.metrics import f1_score, cohen_kappa_score, precision_score, recall_score, classification_report, log_loss, confusion_matrix, accuracy_score 
from sklearn.utils import class_weight
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

In [ ]:
#check Tensorflow Version and GPU availability 
import tensorflow as tf
print(tf.__version__)
#print number of GPUs available to use
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.keras.backend.clear_session(
    free_memory=True
)

In [ ]:
# Code to remove unwanted files and directories like ipython checkpoints and thumbs.db files and keep only the valid image files
import os
import shutil
from PIL import Image  
from PIL import UnidentifiedImageError

# Define the paths for instance and class data directories
train_data_dir = "path/to/train"
val_data_dir = "path/to/val"
test_data_dir = "path/to/test"


# List of unwanted directories and files to look for
unwanted_directories = ['.ipynb_checkpoints']
unwanted_files = ['thumbs.db', 'Thumbs.db', '.DS_Store', 'DS_Store']

def is_valid_image(file_path):
    """
    Function to check if a file is a valid image.
    Returns True if the file is a valid image, False otherwise.
    """
    try:
        img = Image.open(file_path)
        img.verify()  # Verify that it is, in fact, an image
        return True
    except (IOError, SyntaxError, UnidentifiedImageError):
        return False

#clean directories
def clean_directory(directory):
    """
    Function to check for unwanted directories, non-image files, and invalid image files
    in the given directory and delete them.
    """
    # Track the removed files and directories
    removed_items = []

    # Walk through the directory
    for root, dirs, files in os.walk(directory, topdown=False):  # Traverse bottom-up
        # Remove unwanted files and invalid images
        for file_name in files:
            file_path = os.path.join(root, file_name)

            if file_name in unwanted_files:
                print(f"Removing unwanted file: {file_path}")
                os.remove(file_path)  # Remove the unwanted file
                removed_items.append(file_path)
            elif not is_valid_image(file_path):
                print(f"Removing invalid image: {file_path}")
                os.remove(file_path)  # Remove the invalid image file
                removed_items.append(file_path)

        # Remove unwanted directories (now using shutil.rmtree to remove non-empty directories)
        for dir_name in dirs:
            if dir_name in unwanted_directories:
                dir_path = os.path.join(root, dir_name)
                print(f"Removing directory: {dir_path}")
                shutil.rmtree(dir_path)  # Remove the directory and its contents
                removed_items.append(dir_path)

    return removed_items

# Clean the directories
print("Cleaning train_data_dir...")
removed_train_items = clean_directory(train_data_dir)

print("Cleaning val_data_dir...")
removed_val_items = clean_directory(val_data_dir)

print("Cleaning test_data_dir...")
removed_test_items = clean_directory(test_data_dir)

# Output summary of removed items
print(f"\nRemoved {len(removed_train_items)} items from train_data_dir.")
print(f"\nRemoved {len(removed_val_items)} items from val_data_dir.")
print(f"\nRemoved {len(removed_test_items)} items from test_data_dir.")

# If no items were removed
if not removed_train_items and not removed_val_items and not removed_test_items:
    print("No unwanted files, directories, or invalid images were found.")

In [ ]:
#check the number of files in each sub-folder after cleaning

def count_files(directory):
    normal_path = os.path.join(directory, "normal") #class-1
    viral_path = os.path.join(directory, "viral") #class -2
    
    normal_count = len([f for f in os.listdir(normal_path) if os.path.isfile(os.path.join(normal_path, f))])
    viral_count = len([f for f in os.listdir(viral_path) if os.path.isfile(os.path.join(viral_path, f))])    
    return normal_count, viral_count

base_dir = "path/to"
folders = ["train", "val", "test"]

for folder in folders:
    directory = os.path.join(base_dir, folder)
    # Unpack all three values.
    normal_count, viral_count = count_files(directory)
    
    print(f"Number of files in the {folder} directory:")
    print(f"  normal: {normal_count}")
    print(f"  viral: {viral_count}")
    total = normal_count + viral_count
    print(f"  Total: {total}")
    print()

Custom function to plot confusion matrix and print other metrics including confidence intervals for the MCC metric


In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, cohen_kappa_score

def comp_metrix_binary(y_true, y_pred, beta=1, confidence=0.95, conf_vis="both",
                         annot_fontsize=12, tick_fontsize=12, title_fontsize=14,class_names=["Normal", "Viral"]):
    # Convert one-hot to integer labels if needed.
    if np.ndim(y_true) > 1 and y_true.shape[1] > 1:
        y_true_int = np.argmax(y_true, axis=1)
    else:
        y_true_int = np.array(y_true)
    if np.ndim(y_pred) > 1 and y_pred.shape[1] > 1:
        y_pred_int = np.argmax(y_pred, axis=1)
    else:
        y_pred_int = np.array(y_pred)
    
    # Compute default scikit learn based confusion matrix (rows=Actual, cols=Predicted)
    cm_default = confusion_matrix(y_true_int, y_pred_int, labels=[0, 1])
    # Binary: [[TN, FP],
    #          [FN, TP]]
    TN = cm_default[0, 0]
    FP = cm_default[0, 1]
    FN = cm_default[1, 0]
    TP = cm_default[1, 1]
    
    # Clinician's view: transpose of default
    cm_clinician = cm_default.T
    
    # Visualization based on flag
    if conf_vis in ["both", "default", "clinician"]:
        if conf_vis == "both":
            fig, axes = plt.subplots(1, 2, figsize=(12, 5),dpi=300)
            # Default view heatmap
            sns.heatmap(cm_default, annot=True, fmt='d', cmap='cividis', ax=axes[0],
                        xticklabels=class_names, yticklabels=class_names,
                        annot_kws={"size": annot_fontsize})
            axes[0].set_title("Default (Rows=Actual, Cols=Predicted)", fontsize=title_fontsize)
            axes[0].set_xlabel("Predicted Label", fontsize=tick_fontsize)
            axes[0].set_ylabel("Actual Label", fontsize=tick_fontsize)
            axes[0].tick_params(labelsize=tick_fontsize)
            # Clinician's view heatmap
            sns.heatmap(cm_clinician, annot=True, fmt='d', cmap='Greens', ax=axes[1], 
                        xticklabels=class_names, yticklabels=class_names,
                        annot_kws={"size": annot_fontsize})
            axes[1].set_title("Clinician's View (Rows=Predicted, Cols=Actual)", fontsize=title_fontsize)
            axes[1].set_xlabel("Actual Label", fontsize=tick_fontsize)
            axes[1].set_ylabel("Predicted Label", fontsize=tick_fontsize)
            axes[1].tick_params(labelsize=tick_fontsize)
        else:
            fig, ax = plt.subplots(1, 1, figsize=(6, 5),dpi=300)
            if conf_vis == "default":
                sns.heatmap(cm_default, annot=True, fmt='d', cmap='cividis', ax=ax,
                            xticklabels=class_names, yticklabels=class_names,
                            annot_kws={"size": annot_fontsize})
                ax.set_title("Default (Rows=Actual, Cols=Predicted)", fontsize=title_fontsize)
                ax.set_xlabel("Predicted Label", fontsize=tick_fontsize)
                ax.set_ylabel("Actual Label", fontsize=tick_fontsize)
            else:  # clinician view
                sns.heatmap(cm_clinician, annot=True, fmt='d', cmap='Greens', ax=ax, 
                            xticklabels=class_names, yticklabels=class_names,
                            annot_kws={"size": annot_fontsize})
                ax.set_title("Clinician's View (Rows=Predicted, Cols=Actual)", fontsize=title_fontsize)
                ax.set_xlabel("Actual Label", fontsize=tick_fontsize)
                ax.set_ylabel("Predicted Label", fontsize=tick_fontsize)
            ax.tick_params(labelsize=tick_fontsize)
        plt.tight_layout()
        plt.savefig("confusion_matrix.png")
        plt.show()
    
    Population = TN + FP + FN + TP
    Accuracy = (TP + TN) / Population if Population != 0 else float('nan')
    Recall = TP / (TP + FN) if (TP + FN) != 0 else float('nan')  #Recall/Sensitivity
    Precision = TP / (TP + FP) if (TP + FP) != 0 else float('nan')  # PPV
    Specificity = TN / (TN + FP) if (TN + FP) != 0 else float('nan')
    F1 = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) != 0 else float('nan')
    bal_acc = (Recall + Specificity) / 2
    J = Recall + Specificity - 1
    MCC_num = (TP * TN) - (FP * FN)
    MCC_den = math.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN))
    MCC = MCC_num / MCC_den if MCC_den != 0 else float('nan')
    Kappa_den = (TP*FN + TP*FP + 2*TP*TN + FN**2 + FN*TN + FP**2 + FP*TN)
    Kappa = 2 * (TP * TN - FN * FP) / Kappa_den if Kappa_den != 0 else float('nan')
    
    # Wilson binomial confidence interval for Accuracy:
    z = 1.96
    n = Population
    p = MCC
    lower_bound = (p + (z*z)/(2*n) - z * math.sqrt((p*(1-p) + (z*z)/(4*n))/n))/(1 + (z*z)/n) if n > 0 else float('nan')
    upper_bound = (p + (z*z)/(2*n) + z * math.sqrt((p*(1-p) + (z*z)/(4*n))/n))/(1 + (z*z)/n) if n > 0 else float('nan')
    
    FBeta = (1+beta**2) * (Precision * Recall) / ((beta**2 * Precision) + Recall) if ((beta**2 * Precision) + Recall) != 0 else float('nan')
    NPV = TN / (TN + FN) if (TN + FN) != 0 else float('nan')
    
    mat_met = pd.DataFrame({
       'Metric': ['TP', 'TN', 'FP', 'FN', 
                  'Accuracy', 'Recall/Sensitivity', 'Balanced Accuracy', 'Youden', 'Specificity', 'Precision/PPV', 'NPV', 'F1', 'FBeta', 'MCC', 'Kappa',
                  'Wilson_LowerBound', 'Wilson_UpperBound'],
       'Value': [TP, TN, FP, FN, 
                 round(Accuracy,4), round(Recall,4), round(bal_acc,4), round(J,4), round(Specificity,4), round(Precision,4),
                 round(NPV,4), round(F1,4), round(FBeta,4), round(MCC,4), round(Kappa,4),
                 round(lower_bound,4), round(upper_bound,4)]
    })
    
    print("\nComputed Evaluation Metrics (Binary):")
    print(mat_met.to_string(index=False))
    
    return mat_met

In [ ]:
# load data for training,  validation, and testing

"""
The input shape for a Keras model in TensorFlow is typically specified as (img_height, img_width, 3), where:
img_height refers to the number of rows (height or vertical dimension).
img_width refers to the number of columns (width or horizontal dimension).
3 refers to the number of color channels (e.g., RGB).
"""
img_height, img_width = 168, 224

train_data_dir = "path/to/train"
val_data_dir = "path/to/val"
test_data_dir = "path/to/test"

epochs = 64
batch_size = 32
num_classes = 2
input_shape = (img_height, img_width, 3)
model_input = Input(shape=input_shape)
print(model_input)

In [ ]:
#declare data generators

train_datagen = ImageDataGenerator(
    rescale=1./255)
val_datagen = ImageDataGenerator(
        rescale=1./255)
test_datagen = ImageDataGenerator(
        rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='categorical')
val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = False,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size, 
        shuffle = False,
        class_mode='categorical')

#identify the number of samples
nb_train_samples = len(train_generator.filenames)
nb_val_samples = len(val_generator.filenames)
nb_test_samples = len(test_generator.filenames)

#check the class indices
print(train_generator.class_indices)
print(val_generator.class_indices)
print(test_generator.class_indices)

Y_val=val_generator.classes
print(Y_val.shape)
Y_val1=to_categorical(Y_val,num_classes=num_classes)
print(Y_val.shape)
Y_test=test_generator.classes
print(Y_test.shape)
Y_test1=to_categorical(Y_test,num_classes=num_classes)
print(Y_test1.shape)
train_steps = math.ceil(nb_train_samples / batch_size)
val_steps = math.ceil(nb_val_samples / batch_size)
test_steps = math.ceil(nb_test_samples / batch_size)
print(train_steps)
print(val_steps)
print(test_steps)

In [ ]:
#declare model architecture

from tensorflow.keras.applications import VGG16

def create_model():
    # Load the pre-trained model, excluding its top
    model_base = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
    x = model_base.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=model_base.input, outputs=predictions, name='normal_viral_model')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

In [ ]:
# Compute class weights (using scikit-learn)

cw = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(cw))
print("Class weights:", class_weights)

In [ ]:
#declare model checkpoint and store whenever validation loss decreases
# In Keras 3, the recommended extension for saved models is now .keras.

if not os.path.exists("weights"):
    os.makedirs("weights")
checkpoint_filepath = os.path.join("weights", model.name + '.{epoch:02d}-{val_accuracy:.4f}.keras')

checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_loss',
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
    mode='min',
    save_freq='epoch'
)
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5,
    verbose=1, mode='min', min_lr=0.00001
)
callbacks_list = [checkpoint, earlyStopping, reduce_lr]

In [ ]:
# Clear session, reset generators, and begin training
import time

tf.keras.backend.clear_session()
train_generator.reset()
val_generator.reset()
test_generator.reset()

epochs = epochs
t0 = time.time()
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=callbacks_list,
    validation_steps=val_steps,
    verbose=1,
    class_weight=class_weights)
print('Training time: {:.4f} seconds'.format(time.time() - t0))

with open('train_history/normal-viral_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
# plot performance

N = len(history) 
plt.style.use("ggplot")
plt.figure(figsize=(20,10), dpi=40)
plt.plot(np.arange(1, N+1), 
         history.history["loss"], 'orange', label="train_loss")
plt.plot(np.arange(1, N+1), 
         history.history["val_loss"], 'red', label="val_loss")
plt.plot(np.arange(1, N+1), 
          history.history["accuracy"], 'blue', label="train_acc")
plt.plot(np.arange(1, N+1), 
         history.history["val_accuracy"], 'green', label="val_acc")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower right")

In [ ]:
#load model for inference

K.clear_session()
model = load_model('weights/trained_model.keras',compile=False) #path to the trained model
model.summary()

In [ ]:
#Generate test predictions and display confusion matrix and other evaluation metrics values

test_generator.reset()
custom_y_test = model.predict(test_generator,
                                    test_steps,
                                    verbose=1)
print(custom_y_test.shape)
custom_y_test_label = custom_y_test.argmax(axis=-1)

#compute metrics
results_binary_test = comp_metrix_binary(Y_test, custom_y_test_label, beta=1, confidence=0.95, conf_vis="clinician",
                             annot_fontsize=14, tick_fontsize=12, title_fontsize=16, class_names=["Normal", "Viral"])

In [ ]:
# Plot the PR curve for the test data

precision, recall, thresholds = precision_recall_curve(Y_test, 
                                 custom_y_test[:,1])
fscore = (2 * precision * recall) / (precision + recall)
print("The area under the PR curve is", metrics.auc(recall, precision))
fig=plt.figure(figsize=(15,10), dpi=40)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
pyplot.plot(recall, precision, marker='.', color='green', label='Normal/Viral Model')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()

In [ ]:
# Plot the AUROC for the test data
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, custom_y_test[:,1])
roc_auc = roc_auc_score(Y_test, custom_y_test[:,1])
print("The area under the ROC curve is", roc_auc)
fig = plt.figure(figsize=(15,10), dpi=40)
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
major_ticks = np.arange(0.0, 1.1, 0.20) 
minor_ticks = np.arange(0.0, 1.1, 0.20)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
plt.plot(fpr, tpr, marker='.', color='green', label='Model')
# axis labels
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.legend(loc="lower right", prop={"size":20})
plt.show()

In [ ]:
#UMAP visualization

import umap
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Clear any previous session.
tf.keras.backend.clear_session()

# Load the trained model.
model = load_model('weights/trained_model.keras',compile=False) #path to your final model

# Function to extract embeddings from a given layer (e.g., block5_pool).
def extract_embeddings(model, X_batch):
    feature_layer_model = Model(inputs=model.inputs, outputs=model.get_layer('block5_pool').output)
    return feature_layer_model.predict(X_batch)

# Get feature embeddings and class labels from the test generator.
test_embeddings = []
test_labels = []

for i in range(test_steps):
    X_batch, y_batch = next(test_generator)
    test_embeddings.extend(extract_embeddings(model, X_batch))
    test_labels.extend(y_batch.argmax(axis=1))

test_embeddings = np.array(test_embeddings).reshape(len(test_embeddings), -1)
test_labels = np.array(test_labels)

# Define UMAP hyperparameters to iterate over.
neighbors = [2, 5, 10, 15, 20, 50, 100, 200]
distance = [0.0, 0.1, 0.25, 0.5, 0.8, 0.99]
custom_cmap = ListedColormap(['red', 'blue']) #for the two classes
custom_class_names = ['Normal', 'Viral']

# Tune UMAP hyperparameters and save the plots.
for n in neighbors:
    for d in distance:
        umap_reducer = umap.UMAP(n_components=2, n_neighbors=n, min_dist=d, metric='euclidean', random_state=42)
        umap_embeddings = umap_reducer.fit_transform(test_embeddings)
        
        plt.figure(figsize=(12, 6), dpi=100)
        plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1],
                    c=test_labels, cmap=custom_cmap, alpha=0.9, s=2)
        plt.title(f"UMAP Visualization (n_neighbors: {n}, min_dist: {d})")
        
        # Use the manually specified class names for legend.
        handles = [plt.Line2D([], [], marker='o', linestyle='None', markersize=10,
                               color=custom_cmap(i), label=custom_class_names[i])
                   for i in range(len(custom_class_names))]
        plt.legend(handles=handles, bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
        
        # Save the plot with 400 dpi resolution.
        plt.savefig(f"umap_normal_viral_n{n}_d{d}.png", dpi=400, bbox_inches='tight')
        plt.show()

Entropy analysis: Compute the entropy associated with predicting each class (normal, bacterial, viral) using a trained model (one of normal/bacterial, normal/viral, bacterial/viral)

In [ ]:
K.clear_session() 

import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load model
model = load_model('weights/trained_model.keras',compile=False) #path to the trained model

# Set path to test directories
test_dir_viral = "path/to/test/viral"
test_dir_bacterial = "path/to/test/bacterial"
test_dir_normal = "path/to/test/normal"

img_height, img_width = 168, 224

# Load and preprocess images
def load_and_preprocess_images(test_dir):
    image_files = [f for f in os.listdir(test_dir) 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'))]
    print(f"Found {len(image_files)} images in {test_dir}.")
    
    images = []
    filenames = []
    for img_name in image_files:
        img_path = os.path.join(test_dir, img_name)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img) / 255.0
        images.append(img_array)
        filenames.append(img_name)
        
    images = np.array(images)
    print(f"Loaded images shape: {images.shape}")
    return images, filenames

#custom function to compute entropy associated with the predictions
def compute_entropy_for_test_set(test_dir):
    images, filenames = load_and_preprocess_images(test_dir)
    predictions = model.predict(images, verbose=1)  # shape: (n_samples, 2)
    
    def shannon_entropy(prob_vector):
        eps = 1e-10  # avoid log(0)
        p = np.clip(prob_vector, eps, 1.0)
        return -np.sum(p * np.log2(p))
    
    entropies = np.array([shannon_entropy(prob) for prob in predictions])
    df = pd.DataFrame({
        'image': filenames,
        'entropy': entropies
    })
    return df

# Compute entropy for each test set

df_viral = compute_entropy_for_test_set(test_dir_viral)
df_viral['test_set'] = 'Viral'

df_bacterial = compute_entropy_for_test_set(test_dir_bacterial)
df_bacterial['test_set'] = 'Bacterial'

df_normal = compute_entropy_for_test_set(test_dir_normal)
df_normal['test_set'] = 'Normal'

df_results = pd.concat([df_viral, df_bacterial, df_normal], axis=0).reset_index(drop=True)
print(df_results.head())

# Save to Excel
excel_filename = 'entropy_three_class.xlsx'
df_results.to_excel(excel_filename, index=False)
print(f"Results saved to {excel_filename}")

# Print statistics
total_samples = len(df_results)
for i, test_set_name in enumerate(unique_test_sets):
    df_subset = df_results[df_results['test_set'] == test_set_name]
    count = len(df_subset)
    print(f"\nSummary Statistics for {test_set_name} Test Set:")
    print(f"  Number of samples: {count} out of {total_samples} total samples")
    if count > 0:
        print(f"  Mean entropy: {df_subset['entropy'].mean():.4f}")
        print(f"  Median entropy: {df_subset['entropy'].median():.4f}")
        print(f"  Min entropy: {df_subset['entropy'].min():.4f}")
        print(f"  Max entropy: {df_subset['entropy'].max():.4f}")
        print(f"  1st Quartile (25%): {df_subset['entropy'].quantile(0.25):.4f}")
        print(f"  3rd Quartile (75%): {df_subset['entropy'].quantile(0.75):.4f}")
    else:
        print("  No samples for this test set.")

Computing statistical significance:

Standard error for a “proportion”-style stat 𝑞 is approximated as 𝑞(1−𝑞)/𝑛. Z-score tests how many SEs THE observed 𝑞 is from a null value (e.g.\ 0.5 for quartiles, or 0 for IQR).
In a classical Z-test we compute 𝑍=((observed statistic)−(null value))/(standard error), and then convert 𝑍 into a p-value. 
We treat Q1, median (Q2), and Q3 as “proportion-like” values on a scale of 0 to 1. 
By setting these values = 0.5, we are testing “Is the observed quartile significantly different from the midpoint of the scale (0.5)?”
In the context of entropy (which also ranges from 0 to log𝑁, but is often normalized), 0.5 is a convenient mid-point. 
If the data were perfectly uniform (maximum uncertainty), we’d expect normalized entropy near 1; if very “certain,” we’d expect entropy near 0. Testing against 0.5 checks if the observed Q1/Q2/Q3 is meaningfully above or below that halfway mark.
The IQR (inter-quartile range) is non-negative by definition. By setting null_iqr = 0.0, we’re essentially testing “Is the spread between Q3 and Q1 statistically greater than zero?”
Any non-zero IQR indicates variability, but this formal Z-test asks whether that variability is significantly different from zero given your sample size.

Interpretability: 
We assume 0.5 is the “center” of a normalized 0–1 range. 0.0 is the natural lower bound for a range (IQR can’t be negative).
Hypothesis Framing
For Q1/Q2/Q3:

𝐻0: “Quartile = 0.5”
𝐻𝐴: “Quartile ≠ 0.5”

For IQR:
𝐻0: “IQR = 0” (no spread)
𝐻𝐴: “IQR ≠ 0”

Z-Test Formula: 𝑍stat=(observed−null)/SE
Choosing a null provides the denominator’s reference point, so we compute a meaningful Z-score and corresponding p-value.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib.colors as mcolors
from scipy.stats import norm

def compute_wilson_ci(value, population, z=1.96):
    """
    Computes the Wilson binomial confidence interval for a given statistic.
    Here, 'value' is the statistic (e.g. quartile) computed from the sample,
    and 'population' is the total number of samples.
    """
    if population <= 0:
        return float('nan'), float('nan')
    p = value
    n = population
    sqrt_arg = (p * (1 - p) + (z**2) / (4 * n)) / n
    lower_bound = (p + (z**2) / (2 * n) - z * math.sqrt(max(0, sqrt_arg))) / (1 + (z**2) / n)
    upper_bound = (p + (z**2) / (2 * n) + z * math.sqrt(max(0, sqrt_arg))) / (1 + (z**2) / n)
    return round(lower_bound, 4), round(upper_bound, 4)

def get_contrasting_color(hex_color):
    """
    Returns 'black' if the given hex color is light, else 'white'.
    Uses Matplotlib's color conversion rather than seaborn.utils.hex2color.
    """
    rgb = mcolors.to_rgb(hex_color)
    luminance = 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]
    return "black" if luminance > 0.5 else "white"

def plot_entropy_boxplot(df_results, 
                         target_class="Normal", 
                         custom_labels=None,
                         custom_colors=None,
                         output_filename="entropy_boxplot.png",
                         annot_fontsize=14, 
                         tick_fontsize=12, 
                         title_fontsize=16):
    """
    Create a boxplot of Shannon entropy values for three classes:
    Normal, Bacterial, Viral. The 'df_results' DataFrame must have columns:
      - 'test_set' with values in ['Normal', 'Bacterial', 'Viral']
      - 'entropy' with numeric values.
    Also computes Z-scores and two-tailed p-values for Q1, median, Q3, and IQR,
    and prints significance flags (p<0.05) to the console.
    """
    # Default labels/colors
    if custom_labels is None:
        custom_labels = {"Normal":"Normal","Bacterial":"Bacterial","Viral":"Viral"}
    if custom_colors is None:
        custom_colors = {"Normal":"#3CB371","Bacterial":"#4682B4","Viral":"#FF6347"}

    # Validate target_class
    all_classes = ["Normal","Bacterial","Viral"]
    if target_class not in all_classes:
        raise ValueError(f"target_class must be one of {all_classes}")

    # Plot order so target_class appears first
    if target_class=="Viral":
        desired_order = ["Viral","Bacterial","Normal"]
    elif target_class=="Bacterial":
        desired_order = ["Bacterial","Viral","Normal"]
    else:
        desired_order = ["Normal","Bacterial","Viral"]

    # Gather data, labels, colors
    data, labels, box_colors, median_colors = [], [], [], []
    for cls in desired_order:
        vals = df_results.loc[df_results['test_set']==cls, 'entropy'].dropna().values
        if vals.size > 0:
            data.append(vals)
            labels.append(custom_labels[cls])
            c = custom_colors.get(cls, sns.color_palette("pastel",1)[0])
            box_colors.append(c)
            median_colors.append(get_contrasting_color(c))
        else:
            print(f"Warning: No entropy values found for class: {cls}")

    if not data:
        print("No data to plot for the selected classes.")
        return

    # Create boxplot
    fig, ax = plt.subplots(figsize=(10,7), dpi=400, facecolor='white')
    bp = ax.boxplot(data,
                    patch_artist=True,
                    notch=False,
                    vert=True,
                    showfliers=True,
                    whis=1.5)

    # Style boxes, whiskers, caps, medians, fliers
    for patch, col in zip(bp['boxes'], box_colors):
        patch.set_facecolor(col); patch.set_edgecolor("black"); patch.set_linewidth(1.5)
    for w in bp['whiskers']:
        w.set(color='#8B008B', linewidth=1.5, linestyle=":")
    for cap in bp['caps']:
        cap.set(color='#8B008B', linewidth=2)
    for median, mcol in zip(bp['medians'], median_colors):
        median.set(color=mcol, linewidth=3)
    for flier in bp['fliers']:
        flier.set(marker='D', color='#e7298a', alpha=0.5)

    ax.set_xticklabels(labels, fontsize=tick_fontsize)
    ax.set_xlabel("Predicted Class", fontsize=annot_fontsize)
    ax.set_ylabel("Shannon Entropy", fontsize=annot_fontsize)
    ax.set_title(f"Entropy Boxplot (Target: {custom_labels[target_class]})",
                 fontsize=title_fontsize)

    # Annotate and compute stats
    for i, d in enumerate(data):
        n = len(d)
        if n == 0:
            continue

        # Basic stats
        min_v, max_v = d.min(), d.max()
        q1 = np.percentile(d, 25)
        med = np.median(d)
        q3 = np.percentile(d, 75)
        iqr = q3 - q1

        # Wilson CIs
        q1_lb, q1_ub = compute_wilson_ci(q1, n)
        med_lb, med_ub = compute_wilson_ci(med, n)
        q3_lb, q3_ub = compute_wilson_ci(q3, n)
        iqr_lb, iqr_ub = compute_wilson_ci(iqr, n)

        # 1) Compute standard errors (approximate for "proportion-like" stat)
        se_q1 = math.sqrt(q1*(1 - q1)/n)
        se_med = math.sqrt(med*(1 - med)/n)
        se_q3 = math.sqrt(q3*(1 - q3)/n)
        se_iqr = math.sqrt(se_q1**2 + se_q3**2)

        # 2) Null hypotheses (test vs 0.5 for quartiles, vs 0 for IQR)
        null_q1, null_med, null_q3, null_iqr = 0.5, 0.5, 0.5, 0.0

        # 3) Z-scores
        z_q1 = (q1  - null_q1)/se_q1
        z_med = (med - null_med)/se_med
        z_q3 = (q3  - null_q3)/se_q3
        z_iqr = (iqr - null_iqr)/se_iqr

        # 4) Two-tailed p-values
        p_q1 = 2 * (1 - norm.cdf(abs(z_q1)))
        p_med = 2 * (1 - norm.cdf(abs(z_med)))
        p_q3 = 2 * (1 - norm.cdf(abs(z_q3)))
        p_iqr = 2 * (1 - norm.cdf(abs(z_iqr)))

        # 5) Significance flags
        sig_q1 = "statistically significant" if p_q1 < 0.05 else "not significant"
        sig_med = "statistically significant" if p_med < 0.05 else "not significant"
        sig_q3 = "statistically significant" if p_q3 < 0.05 else "not significant"
        sig_iqr = "statistically significant" if p_iqr < 0.05 else "not significant"

        # 6) Console output
        print(f"\nStatistics for {labels[i]} (n={n}):")
        print(f" Q1 = {q1:.4f} [{q1_lb:.4f}, {q1_ub:.4f}], "
              f"z = {z_q1:.2f}, p = {p_q1:.3f} → {sig_q1}")
        print(f" Median = {med:.4f} [{med_lb:.4f}, {med_ub:.4f}], "
              f"z = {z_med:.2f}, p = {p_med:.3f} → {sig_med}")
        print(f" Q3 = {q3:.4f} [{q3_lb:.4f}, {q3_ub:.4f}], "
              f"z = {z_q3:.2f}, p = {p_q3:.3f} → {sig_q3}")
        print(f" IQR = {iqr:.4f} [{iqr_lb:.4f}, {iqr_ub:.4f}], "
              f"z = {z_iqr:.2f}, p = {p_iqr:.3f} → {sig_iqr}")

        # On‐plot annotations (optional)
        x_base = i + 1.05
        ax.text(x_base, max_v, f"Max: {max_v:.4f}", va='bottom', fontsize=annot_fontsize-2)
        ax.text(x_base, q1, f"Q1 ({sig_q1})", va='center', fontsize=annot_fontsize-2)
        ax.text(x_base, med, f"Med ({sig_med})", va='center', fontsize=annot_fontsize-2)
        ax.text(x_base, q3, f"Q3 ({sig_q3})", va='center', fontsize=annot_fontsize-2)
        ax.text(x_base, q3 + 0.02*(ax.get_ylim()[1]-ax.get_ylim()[0]), f"IQR ({sig_iqr})", va='bottom', fontsize=annot_fontsize-2)
    plt.tight_layout(pad=0.5)
    plt.savefig(output_filename, dpi=400, bbox_inches='tight')
    plt.show()

# Example usage:
# plot_entropy_boxplot(
#     df_results,
#     target_class="Viral",
#     custom_labels={"Normal":"Normal","Bacterial":"Bacterial","Viral":"Viral"},
#     custom_colors={"Normal":"#3CB371","Bacterial":"#4682B4","Viral":"#FF6347"},
#     output_filename="entropy_boxplot_with_ztest.png"
# )

END OF CODE